In [2]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode

In [3]:
pd.set_option('display.max_rows', 500)

In [ ]:
df = pd.read_csv("/home/mila/PycharmProjects/Python-codes/Data_scraping/houses_ready.csv", sep=',', low_memory=False, encoding='utf-8', na_values=' ')
df = df.applymap(str)

for i in df.columns:
    df[i] = df[i].str.replace('=', '')
    df[i] = df[i].str.replace('"', '')
    df[i] = df[i].str.replace('None', '')
    df[i] = df[i].str.replace('nan', '')

df = df.replace(np.nan, '')

del df['Unnamed: 0']
df1 = df['date']
del df['date']
df['all'] = df[df.columns].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
df = df['all'].to_frame()
df = pd.concat([df1,df], axis=1)
df.to_excel("10_days1.xlsx")
# len(df[df.num.notnull()]) 
#dictionary = {'Ã': 'a'}
#df = df.replace(dictionary, regex=False, inplace=True)
#df = df.style.set_properties(subset=['all'], **{'width': '300px'})

In [ ]:
#add a column in df with only 3 extracted digits - candidates for rents
df['3num'] = df['all'].str.extract(r'(?<!\d)(\d{3})(?!\d)', expand=False)
#form new df with them
df3num = df[df['3num'].notnull()]
#extract all with slash space with space
df3num['slash_space'] = df3num['all'].str.extract(r'\/[^\S](\d{3})(?!\d)', expand=False)
#extract all with slash space
df3num['slash'] = df3num['all'].str.extract(r'\/(?<!\d)(\d{3})(?!\d)', expand=False)
#extract 3 digits, space and zeros
df3num['zeros'] = df3num['all'].str.extract(r'[/0/][/0/][^\S](?<!\d)(\d{3})(?!\d)', expand=False)
#delete prices with 3 digits, space and 000
re_express = re.compile('([\d]{3})[^\S][/0/][/0/][/0/].*')
df3num['all'] = df3num['all'].str.replace(re_express, '')
#find all with kr . 3dig
df3num['kr'] = df3num['all'].str.extract(r'[/kr/][^\S][/.][^\S](\d{3})(?!\d)', expand=False)
#find all with kı . 3dig
df3num['ki'] = df3num['all'].str.extract(r'[/kı/][^\S][/.][^\S](\d{3})(?!\d)', expand=False)
#find leie 3dig - choose others too - why?
df3num['leie'] = df3num['all'].str.extract(r'[/leie/][^\S](\d{3})(?!\d)', expand=False)
#match rent to one column
df3num['rent'] = df3num['slash']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['kr']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['ki']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['leie']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['zeros']
dfr = df3num[pd.notnull(df3num['rent'])].reset_index(drop=True)

In [ ]:
#dfr = dfr.style.set_properties(subset=['all'], **{'width': '300px'})
# extract rooms
dfr['rooms'] = dfr['all'].str.split('var | v | rom | vär | vær | roms | vor | ettroms | V. | r . | vaer | sier | ver | aer | 2 vier . mod . v Spikkestad . Innsk . 15000 300                                     
').str[0]
# replace Hybelleil as one room
dfr.loc[dfr['rooms'].str.contains('Hybelleil'), 'rooms'] = '1'
# keep only number of rooms
dfr['rooms'] = dfr['rooms'].str.split('(?<!\d)(\d{1})(?!\d)').str[1]
#extract kitchen
dfr['kitchen'] = 0
dfr.loc[dfr['all'].str.contains('grovkjøkken | kjk | kjokken | kj | k \ | Kjokken )'), 'kitchen'] = 1
#extract hall
dfr['hall'] = 0
dfr.loc[dfr['all'].str.contains('hall'), 'hall'] = 1
#extract garage
dfr['garage'] = 0
dfr.loc[dfr['all'].str.contains('garasje'), 'garage'] = 1
#extract bathroom
dfr['bathroom'] = 0
dfr.loc[dfr['all'].str.contains('bad'), 'bathroom'] = 1
#extract balkony
dfr['balkony'] = 0
dfr.loc[dfr['all'].str.contains('balk'), 'balkony'] = 1

In [ ]:
# create fin dataset
dfall = dfr[['date', 'all', 'rent', 'rooms', 'kitchen', 'hall', 'garage', 'bathroom', 'balkony']]
# encode norwegian
for i in range(0,len(dfall)):
    dfall.iloc[i,1] = unidecode(dfall.iloc[i,1])
# extract first Uppercase + 11 characters
dfall['area1'] = dfall['all'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area1'] = dfall['area1'].str.split(r'[^\S]').str[0]
dfall['area1'] = dfall['area1'].replace(np.nan, '', regex=True)
# extract second Uppercase + 11 characters
dfall['second'] = dfall['all']
for i in range(0,len(dfall)):
    dfall.iloc[i, 10] = dfall.iloc[i, 10].replace(dfall.iloc[i, 9],'')
dfall['area2'] = dfall['second'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area2'] = dfall['area2'].replace(np.nan, '', regex=True)
dfall.loc[dfall['area2'].str.contains('St'), 'area2'] = "Hansfjellet"
dfall['area2'] = dfall['area2'].str.split(r'[^\S]').str[0]
#extract third Uppercase + 11 characters
dfall['third'] = dfall['second']
for i in range(0,len(dfall)):
    dfall.iloc[i, 12] = dfall.iloc[i, 12].replace(dfall.iloc[i,11],'')
dfall['area3'] = dfall['third'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area3'] = dfall['area3'].replace(np.nan, '', regex=True)
dfall['area3'] = dfall['area3'].str.split(r'[^\S]').str[0]
#extract fourth Uppercase + 11 characters
dfall['fourth'] = dfall['third']
for i in range(0,len(dfall)):
    dfall.iloc[i, 14] = dfall.iloc[i, 14].replace(dfall.iloc[i,13],'')
dfall['area4'] = dfall['fourth'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area4'] = dfall['area4'].replace(np.nan, '', regex=True)
dfall['area4'] = dfall['area4'].str.split(r'[^\S]').str[0]
#dfall = dfall.style.set_properties(subset=['second'], **{'width': '300px'})
dfall = dfall.drop(['second', 'third', 'fourth', 'all'], axis = 1) 
#replace all area names that are certainly not areas
areas = ['area1', 'area2', 'area3', 'area4']
for i in range(0,4):
    dfall = dfall.replace({areas[i] : { 'Hybelleil' : '', 'Leie' : '', 'Innsk' : '', 'Bm' : '', 'Juli' : '', 'Obligasjon' : '', 'Innsk./leie' : '', 'Kjokken' : '', 'Lele' : '', 'Bis' : ''}})